In [23]:
import requests
import bs4
from bs4 import BeautifulSoup
from langchain_community.document_loaders import WebBaseLoader
import concurrent.futures
import pandas as pd
import openai
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [12]:
def get_soup_from_url(url):
    """
    주어진 URL에서 HTML 페이지를 가져와 BeautifulSoup 객체로 반환하는 함수.
    페이지를 가져오는 데 실패하면 None을 반환.

    Args:
    - url (str): 가져올 웹 페이지의 URL

    Returns:
    - soup (BeautifulSoup): 성공 시 BeautifulSoup 객체, 실패 시 None
    """
    agent = 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    response = requests.get(url, headers={'User-agent': agent})

    if response.status_code != 200:
        print(f"Failed to retrieve page. Status code: {response.status_code}")
        return None
    
    # 성공적으로 페이지를 가져온 경우, BeautifulSoup 객체 생성 및 반환
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [13]:
def extract_news_data(div_list, data, news_link):
    """
    주어진 HTML div 요소 리스트에서 뉴스 기사의 제목, 언론사를 추출하여 
    제공된 리스트에 추가하는 함수.

    Args:
    - div_list (list): 각 뉴스 기사가 포함된 HTML div 요소 리스트.
    - data (list): 기사 제목과 언론사 정보를 담을 리스트.
    - news_link (list): 기사 링크를 담을 리스트.

    Returns:
    - None: 함수는 입력된 리스트를 직접 수정하여 반환하지 않음.
    """
    
    # div_list 안의 각 뉴스 아이템을 순회
    for item in div_list:
        
        # 기사 링크 추출
        link = item.find('a')['href']
        # 기사 제목 추출
        title = item.find('strong', class_='sa_text_strong').text.strip()
        # 언론사 정보 추출
        agency = item.find('div', class_='sa_text_press').text.strip()

        article_data = {'agency': agency, 'title': title}
        data.append(article_data)
        news_link.append(link)

In [39]:
def summarize(text):
    """
    주어진 텍스트를 한국어로 최대 7문장으로 요약하는 함수.
    
    프롬프트 템플릿을 사용하여 주어진 텍스트를 한국어로 7문장 이내로 요약 요청.
    OpenAI의 GPT-3.5-turbo-0125 모델을 이용하여 텍스트 요약을 생성.
    생성된 요약문을 문자열로 반환.
    
    Args:
    - text (str): 요약할 텍스트.

    Returns:
    - str: 요약된 텍스트.
    
    """
    prompt_template = """ Please write a summary within 7 sentences of the following text in Korean Hanguel (한글): 
    "{text}"
    SUMMARY:"""
    prompt = PromptTemplate.from_template(prompt_template)

    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

    output_parser = StrOutputParser()
    
    llm_chain = prompt | llm | output_parser

    response = llm_chain.invoke({"text": text})
    
    return response


In [46]:
def load_web_content(url):
    """
    주어진 URL에서 웹 콘텐츠를 로드하고 특정 부분을 파싱하여 요약된 본문 내용을 반환하는 함수.

    WebBaseLoader를 사용해 주어진 URL의 웹페이지에서 HTML 요소를 찾아 로드.
    가져온 웹페이지 콘텐츠를 summarize 함수를 사용해 요약.
    
    Args:
    - url (str): 웹페이지의 URL 주소.

    Returns:
    - str: 요약된 본문 내용.
    
    """
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                attrs={"class": ['go_trans _article_content']}
            )
        ),
    )

    docs = loader.load()  # 대기 시간 설정
    content = summarize(docs)

    return content


In [47]:
def get_naver_finance_news(date):
    news_links = []
    # 데이터프레임으로 변환할 데이터 딕셔너리 초기화
    data = []

    categories = [258, 259, 260, 261, 262, 263, 771, 310] #경제 뉴스안에서 세부 카테고리

    # 카테고리마다 뉴스 링크 크롤링
    for category in categories:
        base_url = f'https://news.naver.com/breakingnews/section/101/{category}?date={date}'
        
        # URL에서 BeautifulSoup 객체 가져오기
        soup = get_soup_from_url(base_url)
        
        # 페이지를 가져오지 못했을 때
        if soup is None:
            continue

        # 뉴스 링크가 있는 'div' 태그 탐색 (class="sa__text")
        news_divs = soup.find_all('div', class_="sa_text")
        extract_news_data(news_divs,data,news_links)

    return data, news_links

In [19]:
date = "20240910" # 오늘 날짜

In [20]:
data, links = get_naver_finance_news(date)

In [40]:
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    contents = list(executor.map(load_web_content, links))

In [41]:
contents

['애플은 유럽사법재판소(ECJ)의 판결에 따라 130억 유로의 세금을 아일랜드 정부에 되돌려줘야 하며, 이에 따라 최대 100억 달러의 납세 부담이 예상된다. 애플은 이에 대해 미 증권거래소(SEC) 공시를 통해 3분기에 약 100억 달러 규모의 일회성 법인소득세 부과가 이뤄질 것으로 예상된다고 밝혔다. 유럽연합(EU) 최고법원인 ECJ는 아일랜드 정부가 애플에 불법적 법인세 혜택을 제공했다는 판단이 유효하다고 최종 판결을 내렸다. 이에 따라 아일랜드는 애플에 제공한 조세 혜택이 불법 보조금에 해당한다면 세금으로 회수해야 한다는 명령을 받았다. 뉴욕증시에서는 ECJ 판결 소식에 따라 애플 주식이 1% 안팎의 약세를 보였다.',
 '웰스어드바이저스, 현대건설, 마스턴, 메리츠 등이 서울시 강남 르메르디앙 호텔 부지를 복합시설로 개발하는 사업에 참여하고 있으며, 대출 8800억원은 이달 27일에 만기를 맞는다. 이 사업은 도미니크 페로가 설계한 건물로, 서울시 건축심의를 앞두고 사전검토 예정이다. 건축물은 업무, 상업, 숙박시설 등으로 구성되어 있고, 현대건설은 대출약정금의 130%를 연대보증으로 약정했다. LS증권은 유동화증권 매입보장기관으로 활동하며, 유동화증권의 매입보장약정을 체결했다. 이 사업은 서울시 건축심의를 마치고 착공 예정일이 확정될 예정이다.',
 '미국 뉴욕증시의 3대 지수가 혼조세를 보이며 대선 TV 토론과 인플레이션 지표를 대기하고 있다. 다우 평균 지수는 하락하고 S&P500지수와 나스닥지수는 상승하고 있다. 오라클의 실적 발표로 주가가 상승하고, 애플은 하락하고 있다. FOMC 회의를 대기하며 물가 지수에 관심을 가지고 있다. 대선 후보들의 TV 토론으로 변동성이 예상되고, 증시 변동성이 확대될 것으로 예상된다. 국채 금리는 보합세를 유지하고 국제유가는 하락 중이다.',
 '대신증권의 분석에 따르면, 코스피는 매년 9월에 유동성 위축과 VIX 지수의 급등으로 약세장을 보이고 있었으나, 코로나 팬데믹 이후 코스피의 약세가 두드러지며 급락했

In [42]:
data

[{'agency': '연합뉴스', 'title': '애플 "EU 세제혜택 반환 판결 납세부담 13조원대"'},
 {'agency': '이데일리', 'title': "'교보타워 1.4배' 강남 르메르디앙 개발, 이달말 대출 8800억 만기"},
 {'agency': '아시아경제', 'title': '뉴욕증시, 장 초반 혼조…대선 TV 토론·물가 지표 앞두고 관망세'},
 {'agency': '매일경제', 'title': '이 맘때면 덩치 키우는 증시 불청객…선진국도 신흥국도 ‘잔인한 9월’'},
 {'agency': '디지털타임스', 'title': '뉴욕 증시 `관망세`로…엔비디아는 상승세 이어가'},
 {'agency': '이코노미스트', 'title': '“전 세계가 주목” 해리스·트럼프, 첫 TV토론…진검승부 예고'},
 {'agency': '문화일보',
  'title': '“공모주 줄게” 언론사 사칭 주식리딩방 투자자 유인…22억 가로챈 일당[금주머니TV]'},
 {'agency': '디지털타임스', 'title': '신한금융, 정상혁 은행장 등 자회사 CEO 승계절차 개시'},
 {'agency': '이코노미스트', 'title': '일론 머스크 몸매 비결이라던 ‘이 약’, 내달 중순 韓 상륙한다'},
 {'agency': '서울경제', 'title': '추석 전 마지막 공모주…아이언디바이스 청약 경쟁률 1950대1'},
 {'agency': '서울경제', 'title': '패딩가격 100만원 훌쩍… 3040 겨냥 프리미엄화'},
 {'agency': '매일경제', 'title': '시의적절한 의제, 한국 미래 고민'},
 {'agency': 'MBN', 'title': '윤 대통령 "다양한 경험과 지혜, 공존의 디딤돌"…세계지식포럼 개막'},
 {'agency': 'MBN', 'title': '돌고 돌아 "은행 자율로"…이복현 사과했지만, 소비자 혼란은 여전'},
 {'agency': 'MBN', 'title': "추석연휴 인천공항 하루 20만 

In [43]:
df = pd.DataFrame(data)
df['summary'] = contents

In [44]:
df

,agency,title,summary
0,연합뉴스,"애플 ""EU 세제혜택 반환 판결 납세부담 13조원대""",애플은 유럽사법재판소(ECJ)의 판결에 따라 130억 유로의 세금을 아일랜드 정부에...
1,이데일리,"'교보타워 1.4배' 강남 르메르디앙 개발, 이달말 대출 8800억 만기","웰스어드바이저스, 현대건설, 마스턴, 메리츠 등이 서울시 강남 르메르디앙 호텔 부지..."
2,아시아경제,"뉴욕증시, 장 초반 혼조…대선 TV 토론·물가 지표 앞두고 관망세",미국 뉴욕증시의 3대 지수가 혼조세를 보이며 대선 TV 토론과 인플레이션 지표를 대...
3,매일경제,이 맘때면 덩치 키우는 증시 불청객…선진국도 신흥국도 ‘잔인한 9월’,"대신증권의 분석에 따르면, 코스피는 매년 9월에 유동성 위축과 VIX 지수의 급등으..."
4,디지털타임스,뉴욕 증시 `관망세`로…엔비디아는 상승세 이어가,8월 미국 소비자물가(CPI)와 생산자물가(PPI) 공개를 앞두고 미국 증시가 혼조...
...,...,...,...
283,연합뉴스,세종전통시장 방문한 연원정 인사혁신처장,연원정 인사혁신처장이 추석 명절을 앞둔 10일 세종시 조치원읍 세종전통시장에서 사회...
284,연합뉴스,세종전통시장 방문한 연원정 인사혁신처장,연원정 인사혁신처장이 추석 명절을 앞둔 10일 세종시 조치원읍 세종전통시장에서 사회...
285,서울경제,도넘은 K뷰티 유통 경쟁 [기자의 눈],이번 달 6일 서울 성수동에서 열린 '무신사 뷰티 페스타' 행사에는 많은 젊은 방문...
286,동아일보,"강호동 농협회장, ‘사랑의 쌀 꾸러미’ 전달",농협중앙회가 서울 용산구와 성북구에서 '사랑의 쌀 꾸러미 전달 행사'를 개최했다. ...


In [45]:
df.to_csv('news_content4.csv', index=False)